In [1]:
import os
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
import warnings
import glob 
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Desactiva las advertencias
warnings.filterwarnings('ignore')

In [2]:
# Definir las columnas de códigos y de diagnósticos en texto libre
columnas_codigos = ['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3',
                    'COD_DIAGNOSTICO4', 'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6',
                    'COD_DIAGNOSTICO7', 'COD_DIAGNOSTICO8', 'COD_DIAGNOSTICO9',
                    'COD_DIAGNOSTICO10']
columnas_diagnosticos = ['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3',
                         'DIAGNOSTICO4', 'DIAGNOSTICO5', 'DIAGNOSTICO6',
                         'DIAGNOSTICO7', 'DIAGNOSTICO8', 'DIAGNOSTICO9',
                         'DIAGNOSTICO10']


In [208]:
def cargar_dataframe_ambulatorio(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def cargar_dataframe_ieeh(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def filtro_dataframe_ca_tiroides(dataframe):

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer de pulmón
    patron_cancer_pulmon = r'^C34'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de pulmón
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_pulmon, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de pulmón en texto libre
    #patron_diagnostico_pulmon = re.compile(r'(\bc[aá][n]?(?!rv))\b(?:W+\w+){0,4}\W+\b(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_diagnostico_pulmon = re.compile(r'(\bc[aá][n]?(?!rv)).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)

    patron_tumor_pulmon = re.compile(r'(\btu(?!b)|\btumor).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_carcinoma_pulmon = re.compile(r'(\bcarcinoma|\badenocarcinoma).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_neoplasia_pulmon = re.compile(r'(\bneoplasia).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de pulmón en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_neoplasia_pulmon, na=False) |
        x.str.contains(patron_carcinoma_pulmon, na=False)).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[filtro_codigos | filtro_diagnosticos].copy()

def filtro_dataframe_ca_tiroides_ieeh(dataframe):

    # Definir las columnas de códigos y de diagnósticos en texto libre

    columnas_diagnosticos = ['DIAG1','DIAG2','DIAG3']
    # Llenar los valores NaN con una cadena vacía en las columnas de diagnóstico
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].fillna('')

    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)

    # Patrones para identificar diagnósticos de cáncer de pulmón en texto libre
    patron_diagnostico_pulmon = re.compile(r'(\bc[aá][n]?(?!rv)).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_tumor_pulmon = re.compile(r'(\btu(?!b)|\btumor).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_carcinoma_pulmon = re.compile(r'(\bcarcinoma|\badenocarcinoma).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_neoplasia_pulmon = re.compile(r'(\bneoplasia).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de pulmón en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_neoplasia_pulmon, na=False) |
        x.str.contains(patron_carcinoma_pulmon, na=False)).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_tiroides_urgencias(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['CODDIAGNOSTICO']
    columnas_diagnosticos = ['DESCDIAGNOSTICO']

   # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer de pulmón
    patron_cancer_pulmon = r'^C34'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de pulmón
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_pulmon, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de pulmón en texto libre
    patron_diagnostico_pulmon = re.compile(r'(\bc[aá][n]?(?!rv)).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_tumor_pulmon = re.compile(r'(\btu(?!b)|\btumor).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_carcinoma_pulmon = re.compile(r'(\bcarcinoma|\badenocarcinoma).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_neoplasia_pulmon = re.compile(r'(\bneoplasia).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de pulmón en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_neoplasia_pulmon, na=False) |
        x.str.contains(patron_carcinoma_pulmon, na=False)).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_tiroides_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer de pulmón
    patron_cancer_pulmon = r'^C34'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de pulmón
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_pulmon, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de pulmón en texto libre
    patron_diagnostico_pulmon = re.compile(r'(\bc[aá][n]?(?!rv)).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_tumor_pulmon = re.compile(r'(\btu(?!b)|\btumor).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_carcinoma_pulmon = re.compile(r'(\bcarcinoma|\badenocarcinoma).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)
    patron_neoplasia_pulmon = re.compile(r'(\bneoplasia).*(\bpulm[oó]n|\bpulmonar|\bbroncopulmonar)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de pulmón en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_neoplasia_pulmon, na=False) |
        x.str.contains(patron_carcinoma_pulmon, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def agregar_dv(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    #condicion = (resultado_filtrado['TIPO_IDENTIFICADOR_PACIENTE'] == 'RUN')
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[-1]

    #resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
    resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]

    return resultado_filtrado

def agregar_dv_urgencias(resultado_filtrado):

    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['RUT'].str[-1]

    resultado_filtrado['RUT'] = resultado_filtrado['RUT'].str[:-1]

    return resultado_filtrado

def agregar_dv_inter(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE

    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[-1]

    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[:-1]

    return resultado_filtrado

def obtener_diagnosticos_y_codigos(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR_PACIENTE'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4',
                                'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7','DIAGNOSTICO8',
                                'DIAGNOSTICO9','DIAGNOSTICO10']].values.flatten()
    codigos_diagnostico = df_paciente[['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3', 'COD_DIAGNOSTICO4',
                                       'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6', 'COD_DIAGNOSTICO7','COD_DIAGNOSTICO8',
                                       'COD_DIAGNOSTICO9','COD_DIAGNOSTICO10']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_ieeh(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAG1','DIAG2','DIAG3']].values.flatten()
    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]

    return list(set(diagnosticos))

def obtener_diagnosticos_urgencias(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DESCDIAGNOSTICO']].values.flatten()
    codigos_diagnostico = df_paciente[['CODDIAGNOSTICO']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_inter(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR'] == nro_identificacion]
    diagnosticos = df_paciente[['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']].values.flatten()
    codigos_diagnostico = df_paciente[['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def agregar_diagnosticos_y_codigos(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR_PACIENTE').agg({'NRO_IDENTIFICADOR_PACIENTE': 'first', 'DIAGNOSTICO1': 'first',
                                                            'DIAGNOSTICO2': 'first','DIAGNOSTICO3': 'first','DIAGNOSTICO4': 'first',
                                                            'DIAGNOSTICO5': 'first','DIAGNOSTICO6': 'first','DIAGNOSTICO7': 'first',
                                                            'DIAGNOSTICO8': 'first','DIAGNOSTICO9': 'first','DIAGNOSTICO10': 'first',
                                                            'COD_DIAGNOSTICO1': 'first','COD_DIAGNOSTICO2': 'first', 'COD_DIAGNOSTICO2': 'first',
                                                            'COD_DIAGNOSTICO3': 'first', 'COD_DIAGNOSTICO4': 'first', 'COD_DIAGNOSTICO5': 'first',
                                                            'COD_DIAGNOSTICO6': 'first', 'COD_DIAGNOSTICO7': 'first','COD_DIAGNOSTICO8': 'first',
                                                                'COD_DIAGNOSTICO8': 'first',  'COD_DIAGNOSTICO9': 'first',
                                                                'COD_DIAGNOSTICO10': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR_PACIENTE'].apply(lambda x: obtener_diagnosticos_y_codigos(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR_PACIENTE': 'RUT'})
    return resultados

def agregar_diagnosticos_ieeh(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DIAG1': 'first',
                                                            'DIAG1': 'first','DIAG3': 'first'}).reset_index(drop=True)
    #resultados['CODIGOS_DIAGNOSTICO'] = []
    resultados['DIAGNOSTICOS'] = resultados['RUT'].apply(lambda x: obtener_diagnosticos_ieeh(resultado_filtrado, x))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})
    resultados['CODIGOS_DIAGNOSTICO'] = np.nan
    return resultados

def agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DESCDIAGNOSTICO': 'first',
                                                        'CODDIAGNOSTICO': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['RUT'].apply(lambda x: obtener_diagnosticos_urgencias(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})

    return resultados

def agregar_diagnosticos_y_codigos_inter(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR').agg({'NRO_IDENTIFICADOR': 'first', 'SOSP_DIAG_DESC_1': 'first',
                                                            'SOSP_DIAG_DESC_2': 'first','SOSP_DIAG_DESC_3': 'first','SOSP_DIAG_DESC_4': 'first',
                                                            'SOSP_DIAG_DESC_5': 'first','SOSP_DIAG_COD_1': 'first','SOSP_DIAG_COD_2': 'first',
                                                                      'SOSP_DIAG_COD_3': 'first',
                                                            'SOSP_DIAG_COD_4': 'first', 'SOSP_DIAG_COD_5': 'first'
                                                            }).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR'].apply(lambda x: obtener_diagnosticos_inter(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR': 'RUT'})
    return resultados

def crear_dataframe_ambulatorio(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Pulmón',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['PACIENTE'],
        
        'N° Telefono 1': resultado_filtrado.get('TELEFONO_MOVIL', np.nan),
        'N° Telefono 2':resultado_filtrado.get('TELEFONO_FIJO_1', np.nan),
        'N° Telefono 3': resultado_filtrado.get('TELEFONO_FIJO_2', np.nan),
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
   
    return nuevo_dataframe

def crear_dataframe_ieeh(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Pulmón',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES']+ ' '+ resultado_filtrado['APELL_PATE']+ ' '+ resultado_filtrado['APELL_MATE'] ,
        'N° Telefono 1': resultado_filtrado['TELEFONO'],
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_urgencias(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Pulmón',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente': resultado_filtrado['NOMBRES']+ ' ' + resultado_filtrado['APELL_PATE']+ ' '+ resultado_filtrado['APELL_MATE'] ,
        'N° Telefono 1': resultado_filtrado['TELEFONO'],
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_urgencias(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Pulmón',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES_PACIENTE'] + ' ' + resultado_filtrado['APELLIDOS_PACIENTE'],
        'N° Telefono 1': np.nan,
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_inter(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer de Pulmón',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOM_PAC'] + ' ' + resultado_filtrado['APELLPAT_PAC']+' '+resultado_filtrado['APELLMAT_PAC'],
        'N° Telefono 1': resultado_filtrado['FONO_MOVIL'],
        'N° Telefono 2': resultado_filtrado['FONO_FIJO_1'],
        'N° Telefono 3': resultado_filtrado['FONO_FIJO_2'],
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def safe_convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return value

def merge_ambulatorio(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.
    
    return merged_df 

def merge_urgencia(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def merge_inter(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def filtrar_columnas_relevantes(merged_df):

    merged_df['RUT'] = merged_df['RUT'].astype(str)
    merged_df = merged_df[~(merged_df['RUT'].str.contains('A', case=False, na=False))]
    nuevo_dataframe = merged_df.drop_duplicates(subset='RUT', keep='first')
    nuevo_dataframe = nuevo_dataframe[['Problema de Salud','RUT' ,'DV',
       'Nombre del Paciente', 'N° Telefono 1', 'N° Telefono 2',
       'N° Telefono 3', 'Fecha Proximo Hito', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO',
       'Nombre agenda Medico especialista']]
    nuevo_dataframe = nuevo_dataframe.rename(columns={'Problema de Salud': 'Problema Oncologico'})
    nuevo_dataframe['Nombre garantia'] = np.nan
    nuevo_dataframe= nuevo_dataframe[['Problema Oncologico',
    'RUT',
    'DV',
    'Nombre del Paciente',
    'N° Telefono 1',
    'N° Telefono 2',
    'N° Telefono 3',
    'Nombre garantia',
    'Fecha Proximo Hito',
    'DIAGNOSTICOS',
    'CODIGOS_DIAGNOSTICO',
    'Nombre agenda Medico especialista']]
    return nuevo_dataframe

def crear_dataframe_ges(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'Problema Oncologico': dataframe_tiroides['Problema de salud'],
        'RUT': dataframe_tiroides['RUT'],
        'DV': dataframe_tiroides['DV'],
        'Nombre del Paciente':dataframe_tiroides['Nombre'],
        'N° Telefono 1': dataframe_tiroides['Telefonos.x'],
        'N° Telefono 2': dataframe_tiroides['Telefonos.y'],
        'N° Telefono 3': np.nan,
        'Nombre garantia': dataframe_tiroides['Nombre garantia'],
        'Fecha Proximo Hito': np.nan,
        'DIAGNOSTICOS': np.nan,
        'CODIGOS_DIAGNOSTICO': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
    return nuevo_dataframe_2

def crear_dataframe_ultima_atencion(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'RUT': dataframe_tiroides['RUT'],
        'Fecha Ultima atencion': dataframe_tiroides['HORA_TERMINO_ATENCION'],
        'Profesional Ultima atencion': dataframe_tiroides['ESTAMENTO_PROFESIONAL_ATENCION']+ ' '+
        dataframe_tiroides['NOMBRE_PROFESIONAL_ATENCION']+ ' ' + dataframe_tiroides['DESCRIPCION'] + ' '+
        dataframe_tiroides['ESPECIALIDAD_AGENDA']
    })
    return nuevo_dataframe_2

def proceso_dataframe(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
        resultado_filtrado = agregar_dv(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales 

def proceso_dataframe_ultimo(ruta_archivo):
    dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
    resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
    resultado_filtrado = agregar_dv(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
    merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
    return nuevo_dataframe

def proceso_dataframe_ieeh(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_ieeh(dataframe)
        resultados = agregar_diagnosticos_ieeh(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ieeh(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_urgencias(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_urgencias(dataframe)
        resultado_filtrado = agregar_dv_urgencias(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_urgencias(resultado_filtrado)
        merged_df = merge_urgencia(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_inter(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada'])]
    dataframe = dataframe[~dataframe['NRO_IDENTIFICADOR'].str.contains('AT')]
    # Definir la fecha de inicio para el filtro
    fecha_hoy = datetime.today()
    fecha_inicio = fecha_hoy - timedelta(days=10)

    fecha_inicio = pd.to_datetime(fecha_inicio, format='%d/%m/%Y')
    # Filtrar el DataFrame
    dataframe = dataframe[dataframe['FECHA_SOLIC'] >= fecha_inicio]
    resultado_filtrado = filtro_dataframe_ca_tiroides_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado)
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def ultimo_csv_en_carpeta(ruta):
    # Obtener lista de archivos en la carpeta
    archivos = os.listdir(ruta)
    
    # Filtrar archivos que sean .csv
    archivos_csv = [archivo for archivo in archivos if archivo.endswith('.csv')]
    
    if not archivos_csv:
        return None  # Si no hay archivos .csv en la carpeta
    
    # Ordenar la lista de archivos por fecha de modificación
    archivos_csv.sort(key=lambda x: os.path.getmtime(os.path.join(ruta, x)), reverse=True)
    
    # Devolver la ruta completa del último archivo .csv
    return os.path.join(ruta, archivos_csv[0])

def convertir_fecha(fecha):
    try:
        return pd.to_datetime(fecha, format='%d/%m/%Y %H:%M')
    except ValueError:
        return pd.to_datetime(fecha, format='%d-%m-%y %H:%M')

def obtener_fechas_amb(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        dataframe['DV'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[-1]
        dataframe['RUT'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
        



        resultados_totales.append(dataframe)
        
    dataframe = pd.concat(resultados_totales)

    
    # Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
    endocrinologia_df = dataframe[dataframe['ESPECIALIDAD_AGENDA'].isin(['Cirugía Toráx Adulto', 'Broncopulmonar Adulto'])]
    endocrinologia_df = endocrinologia_df[endocrinologia_df['ESTAMENTO_PROFESIONAL_ATENCION']=='Médico']
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
    endocrinologia_df = endocrinologia_df[~endocrinologia_df['ESTADO_CITA'].isin(['No realizada', 'Anulada'])]
    # Convertir la columna 'HORA_TERMINO_ATENCION' a formato de fecha y hora
    #endocrinologia_df['HORA_TERMINO_ATENCION'] = pd.to_datetime(endocrinologia_df['HORA_TERMINO_ATENCION'], format='mixed')
    endocrinologia_df['HORA_TERMINO_ATENCION'] = endocrinologia_df['HORA_TERMINO_ATENCION'].apply(convertir_fecha)

    # Agrupar por RUT y obtener la última fecha de la hora de atención para cada grupo
    #ultima_fecha_por_rut = endocrinologia_df.groupby('RUT').agg({'HORA_TERMINO_ATENCION': 'max'}).reset_index()

    ultima_fecha_por_rut =  endocrinologia_df.groupby('RUT').agg({
    'HORA_TERMINO_ATENCION': 'max',
    'NOMBRE_PROFESIONAL_ATENCION': 'first',
    'ESTAMENTO_PROFESIONAL_ATENCION': 'first',
    'DESCRIPCION': 'first',
    'ESPECIALIDAD_AGENDA': 'first'
}).reset_index()
    # Fusionar el DataFrame original con el DataFrame resultante para recuperar NOMBRE_PROFESIONAL_ATENCION y ESTAMENTO_PROFESIONAL_ATENCION
    resultado = ultima_fecha_por_rut.merge(endocrinologia_df[['RUT', 'HORA_TERMINO_ATENCION', 'NOMBRE_PROFESIONAL_ATENCION', 'ESTAMENTO_PROFESIONAL_ATENCION','DESCRIPCION']], on=['RUT', 'HORA_TERMINO_ATENCION'], how='left')

    return ultima_fecha_por_rut 

# Ruta archivos

In [209]:
dir_actual = os.getcwd()
dir_archivo_csv = os.path.join(dir_actual,'..','Reporteria.lnk')

In [210]:
import win32com.client

def get_target_path(lnk_file):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(lnk_file)
    return shortcut.Targetpath

# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Reporteria.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_reporteria = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_reporteria)


In [211]:
# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Unidad_Gestion_Del_Paciente.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_ges = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_ges)

# Rutas Históricas 

In [215]:
"""

#################### RUTA ARCHIVOS ##########################################

ruta_archivo_4 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', '202401_Reporte_Ambulatorio_Mensual.csv')
ruta_archivo_5 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '20240401_Reporte_Ambulatorio_Diario.csv')


# Lista de rutas de archivos
rutas_archivos = [ruta_archivo_4, ruta_archivo_5]

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)
# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)





#Reportes Ambulatorios

#agregamos los dias 
for i in range(1,int(day_formatted)+1):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
    rutas_archivos.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
for i in range(1,today.month):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Ambulatorio', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))
    rutas_archivos.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))

for i in range(1,6):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Ambulatorio', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
    rutas_archivos.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Anual', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
  

#Reportes IEEH

rutas_archivos_2 = []
for i in range(1,int(day_formatted)+1):
    numero_formateado = f'{i:02}'
    #rutas_archivos_2.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{numero_formateado}-{month_formatted}-{today.year}-Informe_Diario-IEEE.csv'))
    rutas_archivos_2.append(os.path.join(ruta_reporteria,'Reporte_IEEH','Diario', f'{numero_formateado}-{month_formatted}-{today.year}-Informe_Diario-IEEE.csv'))

for i in range(1,today.month):
    numero_formateado = f'{i:02}'    
    #rutas_archivos_2.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual IEEH', f'{numero_formateado}-{today.year}-Informe_Mensual-IEEE.csv'))
    rutas_archivos_2.append(os.path.join(ruta_reporteria,'Reporte_IEEH','Mensual', f'{numero_formateado}-{today.year}-Informe_Mensual-IEEE.csv'))

for i in range(1,13):
    numero_formateado = f'{i:02}'
    #rutas_archivos_2.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual IEEH', f'{numero_formateado}-{today.year-1}-Informe_Mensual-IEEE.csv'))
    rutas_archivos_2.append(os.path.join(ruta_reporteria,'Reporte_IEEH','Mensual', f'{numero_formateado}-{today.year-1}-Informe_Mensual-IEEE.csv'))

for i in range(1,5):
    numero_formateado = f'{i:02}'
    #rutas_archivos_2.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual IEEH', f'{today.year-i-1}-Informe_Anual-IEEE.csv'))
    rutas_archivos_2.append(os.path.join(ruta_reporteria,'Reporte_IEEH','Anual', f'{today.year-i-1}-Informe_Anual-IEEE.csv'))

#Reportes Urgencias
os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario')
rutas_archivos_3 = []
for i in range(1,int(day_formatted)+1):
    numero_formateado = f'{i:02}'
    #rutas_archivos_3.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{numero_formateado}-{month_formatted}-{today.year}-Urgencias_atenciones.csv'))
    rutas_archivos_3.append(os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario', f'{numero_formateado}-{month_formatted}-{today.year}-Urgencias_atenciones.csv'))

for i in range(1,today.month):
    numero_formateado = f'{i:02}'
    #rutas_archivos_3.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Urgencias', f'{numero_formateado}-{today.year}-urgencias_atenciones-mensual.csv'))
    rutas_archivos_3.append(os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Mensual', f'{numero_formateado}-{today.year}-urgencias_atenciones-mensual.csv'))

for i in range(1,6):
    numero_formateado = f'{i:02}'
    #rutas_archivos_3.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Urgencias', f'{today.year-i}-Urgencias_atenciones_Anual.csv'))
    rutas_archivos_3.append(os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Anual', f'{today.year-i}-Urgencias_atenciones_Anual.csv'))

"""

# Término de Rutas Históricas

In [213]:
patron = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')

archivos = glob.glob(patron)

archivos_ordenados = sorted(archivos, key=os.path.getmtime, reverse = True)

ultimos_10_archivos = archivos_ordenados[:10]

for archivo in ultimos_10_archivos:
    fecha_modificacion = datetime.fromtimestamp(os.path.getmtime(archivo))
    #print(f"{archivo}- modificación: {fecha_modificacion}")

In [214]:


#################### RUTA ARCHIVOS ##########################################

dias_reportes = 10

# Lista de rutas de archivos
rutas_archivos = []

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)
# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


#Reportes Ambulatorios Histórico
ruta_archivo_4 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', '202401_Reporte_Ambulatorio_Mensual.csv')
ruta_archivo_5 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '20240401_Reporte_Ambulatorio_Diario.csv')


# Lista de rutas de archivos
rutas_archivos_hist = [ruta_archivo_4, ruta_archivo_5]
#agregamos los dias 
for i in range(1,int(day_formatted)+1):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
for i in range(1,today.month):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Ambulatorio', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))

for i in range(1,6):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Ambulatorio', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Anual', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
  



#Reportes Ambulatorios


patron_amb = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')

archivos_amb = glob.glob(patron_amb)

archivos_ordenados_amb = sorted(archivos_amb, key=os.path.getmtime, reverse = True)

rutas_archivos = archivos_ordenados_amb[:dias_reportes]



#Reportes IEEH

rutas_archivos_2 = []


patron_ieeh = os.path.join(ruta_reporteria,'Reporte_IEEH','Diario', '*Informe_Diario-IEEE*')

archivos_ieeh  = glob.glob(patron_ieeh )

archivos_ordenados_ieeh  = sorted(archivos_ieeh , key=os.path.getmtime, reverse = True)

rutas_archivos_2 = archivos_ordenados_ieeh[:dias_reportes]


#Reportes Urgencias


rutas_archivos_3 = []


patron_urg = os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario', '*Urgencias_atenciones*')

archivos_urg  = glob.glob(patron_urg)

archivos_ordenados_urg  = sorted(archivos_urg , key=os.path.getmtime, reverse = True)

rutas_archivos_3 = archivos_ordenados_urg[:dias_reportes]





In [216]:
#Reportes SIC

patron_sic = os.path.join(ruta_reporteria,'Reportes_Solicitudes','Mensual', '*Informe_Solicitudes_DerivacionCE*')

archivos_sic  = glob.glob(patron_sic)

archivos_ordenados_sic  = sorted(archivos_sic , key=os.path.getmtime, reverse = True)

rutas_archivos_4 = archivos_ordenados_sic[:1]


# Generar dataframes

In [217]:
# Llamar a la función proceso_dataframe con la lista de rutas de archivos
resultado = obtener_fechas_amb(rutas_archivos_hist)

#resultado = obtener_fechas_amb(rutas_archivos_hist)

resultado_final = proceso_dataframe(rutas_archivos)

resultado_final_2 = proceso_dataframe_ieeh(rutas_archivos_2)

resultado_final_3 = proceso_dataframe_urgencias(rutas_archivos_3)

resultado_final_4 = proceso_dataframe_inter(rutas_archivos_4[0])

In [218]:



#os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')
ruta_del_archivo = os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

In [219]:
# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

# Construir la ruta completa al archivo CSV
dataframe['RUT']= dataframe['RUT'].apply(lambda x : str(x))
dataframe['RUT COMPLETO'] = dataframe['RUT'].astype(str)  + dataframe['DV'].astype(str)
dataframe_tiroides = dataframe[dataframe['Problema de salud'] == 'Cáncer de Pulmón . {Decreto Nro 22/2019}']

dataframe_tiroides["Fecha inicio"]=pd.to_datetime(dataframe_tiroides["Fecha inicio"])
# Ordenar en orden descendente por la columna "FECHA_CITA"
dataframe_tiroides = dataframe_tiroides.sort_values(by='Fecha inicio', ascending=False)


# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"
dataframe_tiroides = dataframe_tiroides.drop_duplicates(subset='RUT', keep='first')

nuevo_dataframe_2 = crear_dataframe_ges(dataframe_tiroides)

resultado_final.append(nuevo_dataframe_2)



#if ultimo_csv:
#    print("La dirección del último archivo .csv en la carpeta es:", ultimo_csv)
    
#    resultado_final.append(proceso_dataframe_ultimo(ultimo_csv))
#    df_concatenated = pd.concat(resultado_final)

#else:
#    print("No se encontraron archivos .csv en la carpeta.")
#    df_concatenated = pd.concat(resultado_final)
 # Concatenar los DataFrames
resultado_final.extend(resultado_final_2)
resultado_final.extend(resultado_final_3)
resultado_final.extend([resultado_final_4])
df_concatenated = pd.concat(resultado_final)






# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"

#consolidated_df = df_concatenated.groupby('RUT').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill')).drop_duplicates().reset_index(drop=True)

#resultado_filtrado_2 = consolidated_df.drop_duplicates(subset='RUT', keep='first')

resultado_filtrado_2 = df_concatenated.drop_duplicates(subset='RUT', keep='first')

# Iterar sobre las filas del primer DataFrame
for index, row in resultado_filtrado_2.iterrows():
    rut = row['RUT']
    
    # Buscar el valor correspondiente en el segundo DataFrame
    nombre_garantia = nuevo_dataframe_2.loc[nuevo_dataframe_2['RUT'] == str(rut), 'Nombre garantia'].values
    
    # Encontrar la fila correspondiente en el primer DataFrame y asignar el valor
    resultado_filtrado_2.loc[resultado_filtrado_2['RUT'] == rut, 'Nombre garantia'] = nombre_garantia[0] if nombre_garantia.size > 0 else None

resultado_filtrado_2['RUT'] = resultado_filtrado_2['RUT'].apply(lambda x: str(x))


In [220]:
#nombre_archivo = 'Total pacientes_nuevos_ca_tiroide.xlsx'  # Nombre del archivo de Excel
#resultado_filtrado_2.to_excel(nombre_archivo, index=False)  # index=False para no incluir el índice en el archivo de Excel


In [221]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
0,Cancer de Pulmón,2286979,5,GUILLERMINA DEL CARMEN MIRANDA,97794707,NaN,NaN,None,NaN,[CANCER PULMONAR],[],NaN
2,Cancer de Pulmón,3044984,3,CARLOS DAVID CISTERNAS QUIÑONES,99053898,22850295,NaN,None,NaN,[Tumor maligno de los bronquios y del pulmon],[C34],NaN
3,Cancer de Pulmón,3599871,3,SONIA DE LAS MERCEDES ESPINOZA,92567580,84283708,996240364.0,None,NaN,[Tumor maligno de los bronquios y del pulmon],[C34],NaN
6,Cancer de Pulmón,3703271,9,ALEJANDRO OSCAR ARMA OLIVARES ARANDA,56987140123,56958564522,56959390157,None,NaN,[Tumor maligno de los bronquios y del pulmon],[C34],NaN
10,Cancer de Pulmón,3705732,0,JUAN SEGUNDO MUÑOZ MÉNDEZ,998823975,NaN,NaN,None,NaN,[Tumor maligno de los bronquios y del pulmon],[C34],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35,Cancer de Pulmón,7068856,5,MARÍA ESTER VILCHES CHÁVEZ,NaN,NaN,NaN,None,NaN,[INSUFICIENCIA RESPIRATORIA AGUDA/CRONICA - TU...,[],NaN
44,Cancer de Pulmón,10059878,7,CARLOS HERNÁN VILLEGAS ZORRICUETA,NaN,NaN,NaN,None,NaN,[TU PULMONAR],[],NaN
47,Cancer de Pulmón,12536188,9,JOHANNA LISSETTE MORA BARAHONA,NaN,NaN,NaN,None,NaN,[CANCER DE MAMA CON METASTASIS PULMONAR],[],NaN
49,Cancer de Pulmón,13456082,7,ERIKA ANDREA OLIVARES ZÚÑIGA,NaN,NaN,NaN,None,NaN,[CA PULMONAR EN CUIDADOS PALIATIVOS ],[],NaN


In [223]:
resultado_filtrado_2 = resultado_filtrado_2.drop_duplicates(subset='RUT', keep='first')

In [228]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
0,Cancer de Pulmón,2286979,5,GUILLERMINA DEL CARMEN MIRANDA,97794707,NaN,NaN,None,NaN,[CANCER PULMONAR],[],NaN
2,Cancer de Pulmón,3044984,3,CARLOS DAVID CISTERNAS QUIÑONES,99053898,22850295,NaN,None,NaN,[Tumor maligno de los bronquios y del pulmon],[C34],NaN
3,Cancer de Pulmón,3599871,3,SONIA DE LAS MERCEDES ESPINOZA,92567580,84283708,996240364.0,None,NaN,[Tumor maligno de los bronquios y del pulmon],[C34],NaN
6,Cancer de Pulmón,3703271,9,ALEJANDRO OSCAR ARMA OLIVARES ARANDA,56987140123,56958564522,56959390157,None,NaN,[Tumor maligno de los bronquios y del pulmon],[C34],NaN
10,Cancer de Pulmón,3705732,0,JUAN SEGUNDO MUÑOZ MÉNDEZ,998823975,NaN,NaN,None,NaN,[Tumor maligno de los bronquios y del pulmon],[C34],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35,Cancer de Pulmón,7068856,5,MARÍA ESTER VILCHES CHÁVEZ,NaN,NaN,NaN,None,NaN,[INSUFICIENCIA RESPIRATORIA AGUDA/CRONICA - TU...,[],NaN
44,Cancer de Pulmón,10059878,7,CARLOS HERNÁN VILLEGAS ZORRICUETA,NaN,NaN,NaN,None,NaN,[TU PULMONAR],[],NaN
47,Cancer de Pulmón,12536188,9,JOHANNA LISSETTE MORA BARAHONA,NaN,NaN,NaN,None,NaN,[CANCER DE MAMA CON METASTASIS PULMONAR],[],NaN
49,Cancer de Pulmón,13456082,7,ERIKA ANDREA OLIVARES ZÚÑIGA,NaN,NaN,NaN,None,NaN,[CA PULMONAR EN CUIDADOS PALIATIVOS ],[],NaN


In [ ]:
resultado_filtrado_2 = resultado_filtrado_2.drop_duplicates(subset='RUT', keep='first')

# Cruce BDD Gestoras

In [112]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA PULMONAR'))

['BASE TRABAJO GESTION ONCOLOGICA_CANCER PULMON.xlsx',
 'Reporte diario casos nuevos']

In [113]:
archivo_excel = os.path.join(dir_actual, '..','CA PULMONAR', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER PULMON.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


In [114]:
dataframe_2

,Problema Oncologico,Rut,DV,Nombre,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer de Pulmón,5812324,2,"RIVEROS RETAMALES, MIGUEL ANGEL",Alta,Fallecido,NaN,NaN,NaN,21-11-2019,NaN,NaN,NaN,NaN,NaN,NaN
1,Cáncer de Pulmón,4763858,5,"TORRES VERA, GLENDA HELVESIA",Alta,Fallecido,NaN,NaN,NaN,2023-10-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,Cáncer de Pulmón,5226563,0,"GATICA MÉNDEZ, VÍCTOR MANUEL",Alta,Fallecido,NaN,NaN,NaN,2019-10-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,Cáncer de Pulmón,5403635,3,"LLANTÉN VENEGAS, FRESIA GEORGINA",Alta,Fallecido,NaN,NaN,NaN,2020-08-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,Cáncer de Pulmón,8550063,5,"CARMONA CORNEJO, JAIME ENRIQUE",Alta,Fallecido,NaN,NaN,NaN,28-06-2023,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,Cancer de Pulmon,9769636,5,MIGUEL PATRICIO MARABOLÍ BARRIENTOS,NaN,NaN,NaN,NaN,NaN,NaN,"['Tumor maligno de los bronquios o del pulmon,...",62835363/09990503607/-,NaN,NaN,NaN,NaN
625,Cancer de Pulmon,9792505,4,DAVID ALEJANDRO ESTRELLA ROSALES,NaN,NaN,NaN,NaN,NaN,NaN,['Examen de pesquisa especial para tuberculosi...,0/0/-,NaN,NaN,NaN,NaN
626,Cancer de Pulmon,9869336,K,MARÍA JIMENA ROJAS PAREDES,NaN,NaN,NaN,NaN,NaN,NaN,"['NEUMONIA ASPIRATIVA', 'Neumonia bacteriana n...",56998424688/56957386942/-,NaN,NaN,NaN,NaN
627,Cancer de Pulmon,9917387,4,ÁLVARO ANTONIO CHACÓN LUENGO,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'],0993112390/02962398593/-,NaN,NaN,NaN,NaN


In [225]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion


In [230]:

dataframe = resultado_filtrado_2.copy()

# Obtener la ruta del directorio actual del script
#dir_actual = os.path.dirname(__file__)
#dir_actual = os.getcwd()

dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA PULMONAR'))

# Construir la ruta completa al archivo CSV
archivo_excel = os.path.join(dir_actual, '..','CA PULMONAR', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER PULMON.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


# Obtener la cantidad de filas que no son nulas en la columna 'mi_columna'
cantidad_filas_no_nulas = dataframe_2['Rut'].count()



print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
def quitar_ultimos_dos(elemento):
    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
for index, fila in filas_dv_nan.iterrows():
    rut = str(fila['RUT'])
    if True:
      dv = rut[-1]  # Último dígito del Rut
      dataframe_2.at[index, 'DV'] = dv
      dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut


dataframe_2 = dataframe_2[~(dataframe_2['RUT'].str.contains('A', case=False, na=False))]

dataframe_2['RUT'] = dataframe_2['RUT'].astype(float)
dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)


Cantidad de pacientes en la base 950


In [232]:
list(dataframe_2.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Fecha de Comité',
 'Observacion',
 'Diagnosticos',
 'Telefonos',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

In [233]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Pulmón,5891114,3,NANCY DEL CARMEN POBLETE MARTÍN,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor de comportamiento incierto o desconoci...,-/-/-,01-08-2024,JOSE RAMON SILVA FIGUEROA,02-02-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
1,Cancer de Pulmón,12112985,K,JOHANNA JUDITH OTEÍZA OLMOS,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'...,56965288411/-/-,02-07-2024,TAC 2,NaN,NaN
2,Cancer de Pulmón,4886097,4,GLORYA DINA GHO ARAYA,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'...,0998258432/-/-,02-07-2024,TAC 2,15-04-2024,Médico PATRICIA MARISOL ARIAS CAMPOVERDE Contr...
3,Cancer de Pulmón,5029280,0,MANUEL PAINEMAL COLIHUINCA,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor de comportamiento incierto o desconoci...,-/-/-,03-07-2024,TAC 2,NaN,NaN
4,Cancer de Pulmón,5896606,1,FRANCISCO SIXTO IBARRA REY,NaN,NaN,NaN,NaN,NaN,NaN,['Hallazgos anormales en diagnostico por image...,994273100/987731525/994273100,03-07-2024,JOSE RAMON SILVA FIGUEROA,13-02-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,Cancer de Pulmón,9869336,K,MAR?A JIMENA ROJAS PAREDES,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'...,56998424688/56957386942/-,NaN,NaN,NaN,NaN
946,Cancer de Pulmón,9892800,6,EUGENIA DEL CARMEN ZAMORANO APABLAZA,NaN,NaN,NaN,NaN,NaN,NaN,['CANCER PULMONAR METASTASICO'],944648925/-/-,NaN,NaN,NaN,NaN
947,Cancer de Pulmón,9917387,4,ÁLVARO ANTONIO CHACÓN LUENGO,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'],0993112390/02962398593/-,NaN,NaN,NaN,NaN
948,Cancer de Pulmón,9962036,6,FRANCISCO ANTONIO SÁEZ GÓMEZ,Alta,Fallecido,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'],-/-/-,NaN,NaN,NaN,NaN


In [234]:
list(dataframe_2.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Fecha de Comité',
 'Observacion',
 'Diagnosticos',
 'Telefonos',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

# Merge Total pacientes vs Gestoras

In [235]:
# Ejemplo de creación de DataFrame (puedes cargar tus propios DataFrames)
df1 = dataframe.copy()
df2 = dataframe_2.copy()

# Agrega una columna para identificar el origen de cada RUT
df1['_df'] = 'df1'
df2['_df'] = 'df2'

# Convertir la columna 'RUT' en ambos DataFrames al mismo tipo de datos
df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)

#print(f"df_1: {df1['RUT'] }")
#print(f"df_2: {df2['RUT'] }")

# Realiza un merge de los dos DataFrames en la columna 'RUT' y conserva solo las filas que aparecen en 'left_only'
df_merge = pd.merge(df1, df2, on='RUT', how='outer', indicator=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Nombre del Paciente_x'].fillna(df_merge['Nombre del Paciente_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Nombre del Paciente_x': 'Nombre del Paciente'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Nombre del Paciente_y'], inplace=True)

# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Problema Oncologico_x'].fillna(df_merge['Problema Oncologico_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Problema Oncologico_x': 'Problema Oncologico'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Problema Oncologico_y'], inplace=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['DV_x'].fillna(df_merge['DV_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'DV_x': 'DV'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['DV_y'], inplace=True)

df_ca_tiroide = df_merge[df_merge['_merge']=='left_only']

# Reemplazar los valores NaN por '-'
df_ca_tiroide['N° Telefono 1'] = df_ca_tiroide['N° Telefono 1'].fillna('-')
df_ca_tiroide['N° Telefono 2'] = df_ca_tiroide['N° Telefono 2'].fillna('-')
df_ca_tiroide['N° Telefono 3'] = df_ca_tiroide['N° Telefono 3'].fillna('-')

def combinar_telefonos(row):
    return '/'.join(filter(None, [str(row['N° Telefono 1']), str(row['N° Telefono 2']), str(row['N° Telefono 3'])]))

# Aplicar la función a lo largo de las filas del DataFrame para crear la columna 'Telefono Consolidado'
df_ca_tiroide_telefono = df_ca_tiroide[['N° Telefono 1','N° Telefono 2','N° Telefono 3']]
df_ca_tiroide['Telefono Consolidado2'] = df_ca_tiroide_telefono.apply(combinar_telefonos, axis=1)

# Eliminar las columnas originales de los números de teléfono si ya no son necesarias
df_ca_tiroide.drop(columns=['N° Telefono 1', 'N° Telefono 2', 'N° Telefono 3'], inplace=True)

df_ca_tiroide['Etapa'] = np.nan

df_ca_tiroide['Estado'] = np.nan

df_ca_tiroide['Fecha Proximo Hito'] = np.nan

df_ca_tiroide['Observacion'] = np.nan

df_ca_tiroide['Medico Responsable'] = np.nan

df_ca_tiroide['Fecha Ultima atencion'] = np.nan

df_ca_tiroide['Profesional Ultima atencion'] = np.nan

df_ca_tiroide['MEDICO'] = np.nan

df_ca_tiroide['CITAS FUTURAS'] = np.nan

#df_ca_tiroide = df_ca_tiroide[['Problema Oncologico', 'RUT', 'DV', 'Nombre del Paciente','Etapa','Nombre garantia','Estado','Fecha Proximo Hito','Observacion','Medico Responsable','Fecha Ultima atencion', 'Profesional Ultima atencion','DIAGNOSTICOS', 'Telefono Consolidado2']]
df_ca_tiroide = df_ca_tiroide[['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Observacion',
 'Medico Responsable',
 'DIAGNOSTICOS',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']]

In [236]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Observacion,Medico Responsable,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion


In [121]:
df_ca_tiroide['Fecha de Comité'] = np.nan

In [122]:
lista = ['Problema Oncologico', 'RUT','DV','Nombre del Paciente','Etapa','Estado','Fecha Próximo Hito','Medico Responsable','Fecha de Comité','Observacion','DIAGNOSTICOS','Telefono Consolidado2','CITAS FUTURAS','MEDICO','Fecha Ultima atencion','Profesional Ultima atencion']

In [123]:
print(f'Cantidad de pacientes nuevos encontrados: {df_ca_tiroide.shape[0]}')

Cantidad de pacientes nuevos encontrados: 793


# Guardar estadística de llegada


In [49]:
fecha_hoy = date.today()
data = {'Problema Oncologico': ['Cancer de Pulmón'], 'Fecha Medición': [fecha_hoy], 'Cantidad de llegada': [df_ca_tiroide.shape[0]], 'Cantidad de pacientes base': [cantidad_filas_no_nulas]}
estadisticas_llegadas = pd.DataFrame(data)

nombre_archivo = f'Estadisticas_llegadas.xlsx'
archivo_excel = os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}')

df_existente = pd.read_excel(archivo_excel)
df_existente['Fecha Medición'] = pd.to_datetime(df_existente['Fecha Medición'])


wb = Workbook()
ws = wb.active
df_final = pd.concat([df_existente,estadisticas_llegadas], ignore_index=True)


for r in dataframe_to_rows(df_final, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'Estadisticas_llegadas.xlsx'
wb.save(os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}'))


# Agregar Fallecidos

In [124]:
dir_actual = os.getcwd()
dir_gestoras = os.path.join(dir_actual,'..')

In [125]:
ruta_del_archivo_2 = os.path.join(dir_gestoras, 'FALLECIDOS HLF CORTE 30_04_2024.xlsx')


In [126]:
# Carga el archivo XLSX en un DataFrame
fallecidos = pd.read_excel(ruta_del_archivo_2)

In [127]:
fallecidos_rut = pd.DataFrame({'RUT': fallecidos['RUD']})

In [128]:
import pandas as pd

# Suponiendo que tus dataframes se llaman df_consolidado y df_fallecidos


# Combinamos los dataframes en función de la columna 'RUT'


fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)  # Convertir la columna a tipo string
fallecidos_rut['RUT'] = fallecidos_rut['RUT'].replace(['-', '\.'], '', regex=True) 

fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)


In [129]:
df_pacientes_nuevos = df_ca_tiroide[lista].copy()

In [130]:
df_pacientes_nuevos


,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Pulmón,10024287,7,ISABEL ALEJANDRA DONOSO MEDRANO,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],0981550027/0975219148/-,NaN,NaN,NaN,NaN
1,Cancer de Pulmón,10059878,7,CARLOS HERNÁN VILLEGAS ZORRICUETA,NaN,NaN,NaN,NaN,NaN,NaN,[TU PULMONAR],-/-/-,NaN,NaN,NaN,NaN
2,Cancer de Pulmón,10088381,3,VÍCTOR JOSÉ MARTÍNEZ CUEVAS,NaN,NaN,NaN,NaN,NaN,NaN,"[SARCOMA EWING, TUMOR DE HUESO PÚBICO CON METÁ...",-/-/-,NaN,NaN,NaN,NaN
3,Cancer de Pulmón,10097136,4,EDUARDO ALBERTO NOHALES LEBRA,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",941057839/-/-,NaN,NaN,NaN,NaN
4,Cancer de Pulmón,10107931,2,MALAQUIA RAMON LEAL AGUILAR,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,Cancer de Pulmón,9792505,4,DAVID ALEJANDRO ESTRELLA ROSALES,NaN,NaN,NaN,NaN,NaN,NaN,[Hallazgos anormales en diagnostico por imagen...,0/0/-,NaN,NaN,NaN,NaN
789,Cancer de Pulmón,9869336,K,MARÍA JIMENA ROJAS PAREDES,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",56998424688/56957386942/-,NaN,NaN,NaN,NaN
790,Cancer de Pulmón,9917387,4,ÁLVARO ANTONIO CHACÓN LUENGO,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],0993112390/02962398593/-,NaN,NaN,NaN,NaN
791,Cancer de Pulmón,9962036,6,FRANCISCO ANTONIO SÁEZ GÓMEZ,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-,NaN,NaN,NaN,NaN


In [131]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Estado'] = 'Fallecido'


In [132]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Etapa'] = 'Alta'


In [133]:
df_pacientes_nuevos

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Pulmón,10024287,7,ISABEL ALEJANDRA DONOSO MEDRANO,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],0981550027/0975219148/-,NaN,NaN,NaN,NaN
1,Cancer de Pulmón,10059878,7,CARLOS HERNÁN VILLEGAS ZORRICUETA,Alta,Fallecido,NaN,NaN,NaN,NaN,[TU PULMONAR],-/-/-,NaN,NaN,NaN,NaN
2,Cancer de Pulmón,10088381,3,VÍCTOR JOSÉ MARTÍNEZ CUEVAS,NaN,NaN,NaN,NaN,NaN,NaN,"[SARCOMA EWING, TUMOR DE HUESO PÚBICO CON METÁ...",-/-/-,NaN,NaN,NaN,NaN
3,Cancer de Pulmón,10097136,4,EDUARDO ALBERTO NOHALES LEBRA,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",941057839/-/-,NaN,NaN,NaN,NaN
4,Cancer de Pulmón,10107931,2,MALAQUIA RAMON LEAL AGUILAR,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,Cancer de Pulmón,9792505,4,DAVID ALEJANDRO ESTRELLA ROSALES,NaN,NaN,NaN,NaN,NaN,NaN,[Hallazgos anormales en diagnostico por imagen...,0/0/-,NaN,NaN,NaN,NaN
789,Cancer de Pulmón,9869336,K,MARÍA JIMENA ROJAS PAREDES,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",56998424688/56957386942/-,NaN,NaN,NaN,NaN
790,Cancer de Pulmón,9917387,4,ÁLVARO ANTONIO CHACÓN LUENGO,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],0993112390/02962398593/-,NaN,NaN,NaN,NaN
791,Cancer de Pulmón,9962036,6,FRANCISCO ANTONIO SÁEZ GÓMEZ,Alta,Fallecido,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-,NaN,NaN,NaN,NaN


In [134]:
df_pacientes_nuevos[df_pacientes_nuevos['Etapa']=='Alta']

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
1,Cancer de Pulmón,10059878,7,CARLOS HERNÁN VILLEGAS ZORRICUETA,Alta,Fallecido,NaN,NaN,NaN,NaN,[TU PULMONAR],-/-/-,NaN,NaN,NaN,NaN
7,Cancer de Pulmón,10154803,1,JUAN CARLOS CARRIÓN IBÁÑEZ,Alta,Fallecido,NaN,NaN,NaN,NaN,"[CA PULMONAR, Tumor maligno de los bronquios y...",995231598/95231598/-,NaN,NaN,NaN,NaN
16,Cancer de Pulmón,10480456,K,JUAN CARLOS GONZÁLEZ AGUILERA,Alta,Fallecido,NaN,NaN,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,96554906/96554906/-,NaN,NaN,NaN,NaN
19,Cancer de Pulmón,10613129,5,JUANA DEL CARMEN NILO TORO,Alta,Fallecido,NaN,NaN,NaN,NaN,[CA PULMONAR METASTASICO EN CUIDADOS PALITAVIOS],98083018/-/-,NaN,NaN,NaN,NaN
22,Cancer de Pulmón,10642073,4,JUAN MARCELO GAJARDO BARRUETO,Alta,Fallecido,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",-/-/-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,Cancer de Pulmón,9214817,3,MARIANELA ELVIRA PAZ CÁRDENAS,Alta,Fallecido,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-,NaN,NaN,NaN,NaN
772,Cancer de Pulmón,9280864,5,SERGIO ENRIQUE LEYTON ZÚÑIGA,Alta,Fallecido,NaN,NaN,NaN,NaN,"[CA PULMONAR, ]",///-/-,NaN,NaN,NaN,NaN
781,Cancer de Pulmón,9575116,4,BERTA ISABEL CASTRO PINEDA,Alta,Fallecido,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],988648037/935132262/-,NaN,NaN,NaN,NaN
786,Cancer de Pulmón,9707792,4,MIREYA MARISOL ULLOA FAÚNDEZ,Alta,Fallecido,NaN,NaN,NaN,NaN,"[, Tumor maligno de los bronquios y del pulmon]",0223012305///-/-,NaN,NaN,NaN,NaN


In [135]:
df_pacientes_nuevos = df_pacientes_nuevos[~(df_pacientes_nuevos['RUT']=='nan')]

# Guardar dataframe

In [136]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA PULMONAR','Reporte diario casos nuevos'))

['20240515_Pacientes_nuevos_ca_pulmonar.xlsx',
 '20240625_Pacientes_nuevos_ca_pulmonar - copia.xlsx',
 '20240625_Pacientes_nuevos_ca_pulmonar.xlsx']

In [137]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_Pacientes_nuevos_ca_pulmonar.xlsx'  # Nombre del archivo de Excel
df_pacientes_nuevos.to_excel(os.path.join(dir_actual, '..','CA PULMONAR','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 

# Convertir la columna 'RUT' en DataFrame_2 a tipo object si es necesario


df_concatenated_2 = pd.concat([dataframe_2,df_pacientes_nuevos])

#print(list[df_concatenated_2.columns])




df_concatenated_2['RUT'] = df_concatenated_2['RUT'].astype(str)
resultado['RUT'] = resultado['RUT'].astype(str)
# Fusionar DataFrame_2 con el DataFrame de resultado utilizando la columna 'RUT' como clave de fusión

dataframe_final = df_concatenated_2.merge(resultado, on='RUT', how='left')

#print(list[dataframe_final.columns])


dataframe_final_2 = crear_dataframe_ultima_atencion(dataframe_final)

In [138]:
df_concatenated_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion,DIAGNOSTICOS,Telefono Consolidado2
0,Cancer de Pulmón,10024287,7,ISABEL ALEJANDRA DONOSO MEDRANO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],0981550027/0975219148/-
1,Cancer de Pulmón,10059878,7,CARLOS HERNÁN VILLEGAS ZORRICUETA,Alta,Fallecido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[TU PULMONAR],-/-/-
2,Cancer de Pulmón,10088381,3,VÍCTOR JOSÉ MARTÍNEZ CUEVAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SARCOMA EWING, TUMOR DE HUESO PÚBICO CON METÁ...",-/-/-
3,Cancer de Pulmón,10097136,4,EDUARDO ALBERTO NOHALES LEBRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",941057839/-/-
4,Cancer de Pulmón,10107931,2,MALAQUIA RAMON LEAL AGUILAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,Cancer de Pulmón,9792505,4,DAVID ALEJANDRO ESTRELLA ROSALES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Hallazgos anormales en diagnostico por imagen...,0/0/-
789,Cancer de Pulmón,9869336,K,MARÍA JIMENA ROJAS PAREDES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",56998424688/56957386942/-
790,Cancer de Pulmón,9917387,4,ÁLVARO ANTONIO CHACÓN LUENGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],0993112390/02962398593/-
791,Cancer de Pulmón,9962036,6,FRANCISCO ANTONIO SÁEZ GÓMEZ,Alta,Fallecido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-


In [139]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,10024287,2023-10-20 13:18:00,Médico EDGARDO JAVIER OSSANDÓN MOLINA CONSULTA...
1,10059878,NaT,NaN
2,10088381,NaT,NaN
3,10097136,2024-03-27 11:05:00,Médico PATRICIA MARISOL ARIAS CAMPOVERDE CONSU...
4,10107931,NaT,NaN
...,...,...,...
788,9792505,2023-11-06 12:40:00,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...
789,9869336,NaT,NaN
790,9917387,NaT,NaN
791,9962036,NaT,NaN


In [140]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%Y-%m-%d')


In [141]:
dataframe_final_2 = dataframe_final_2.dropna()

In [142]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,10024287,2023-10-20,Médico EDGARDO JAVIER OSSANDÓN MOLINA CONSULTA...
3,10097136,2024-03-27,Médico PATRICIA MARISOL ARIAS CAMPOVERDE CONSU...
8,10202270,2024-06-12,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
10,10206062,2022-11-29,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...
11,10238037,2022-03-25,Médico EDGARDO JAVIER OSSANDÓN MOLINA Control ...
...,...,...,...
776,9472638,2024-05-08,Médico JOSE RAMON SILVA FIGUEROA Cuerpo extrañ...
777,9472638,2024-05-08,Médico JOSE RAMON SILVA FIGUEROA Cuerpo extrañ...
781,9575116,2023-07-31,Médico PATRICIA MARISOL ARIAS CAMPOVERDE CONSU...
782,9575179,2024-06-17,Médico PATRICIA MARISOL ARIAS CAMPOVERDE Contr...


In [147]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%d-%m-%Y')


In [148]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,10024287,20-10-2023,Médico EDGARDO JAVIER OSSANDÓN MOLINA CONSULTA...
3,10097136,27-03-2024,Médico PATRICIA MARISOL ARIAS CAMPOVERDE CONSU...
8,10202270,12-06-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
10,10206062,29-11-2022,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...
11,10238037,25-03-2022,Médico EDGARDO JAVIER OSSANDÓN MOLINA Control ...
...,...,...,...
776,9472638,08-05-2024,Médico JOSE RAMON SILVA FIGUEROA Cuerpo extrañ...
777,9472638,08-05-2024,Médico JOSE RAMON SILVA FIGUEROA Cuerpo extrañ...
781,9575116,31-07-2023,Médico PATRICIA MARISOL ARIAS CAMPOVERDE CONSU...
782,9575179,17-06-2024,Médico PATRICIA MARISOL ARIAS CAMPOVERDE Contr...


In [149]:
import pandas
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [179]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(dataframe_final_2, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_pulmon.xlsx'
wb.save(os.path.join(dir_actual, '..','CA PULMONAR',f'{nombre_archivo}'))

# Guardar dataframe ultima atención 

In [151]:
#nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_tiroides.xlsx'  # Nombre del archivo de Excel
#dataframe_final_2.to_excel(os.path.join(dir_actual, '..','CA TIROIDES',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 


In [180]:
df1 = df_concatenated_2.copy()
df2 = dataframe_final_2.copy()

df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)


# Realizar la fusión utilizando la columna 'RUT'
df_merged = pd.merge(df1, df2, on='RUT', how='left')



In [181]:
df2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,10024287,20-10-2023,Médico EDGARDO JAVIER OSSANDÓN MOLINA CONSULTA...
3,10097136,27-03-2024,Médico PATRICIA MARISOL ARIAS CAMPOVERDE CONSU...
8,10202270,12-06-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
10,10206062,29-11-2022,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...
11,10238037,25-03-2022,Médico EDGARDO JAVIER OSSANDÓN MOLINA Control ...
...,...,...,...
776,9472638,08-05-2024,Médico JOSE RAMON SILVA FIGUEROA Cuerpo extrañ...
777,9472638,08-05-2024,Médico JOSE RAMON SILVA FIGUEROA Cuerpo extrañ...
781,9575116,31-07-2023,Médico PATRICIA MARISOL ARIAS CAMPOVERDE CONSU...
782,9575179,17-06-2024,Médico PATRICIA MARISOL ARIAS CAMPOVERDE Contr...


In [182]:
len(df1['RUT'].unique())

781

In [183]:
len(df2['RUT'].unique())

225

In [184]:
# Renombrar columnas para evitar colisiones
df_merged.rename(columns={'Fecha Ultima atencion_y': 'Fecha Ultima atencion', 'Profesional Ultima atencion_y': 'Profesional Ultima atencion'}, inplace=True)

# Eliminar columnas duplicadas
df_merged.drop(['Fecha Ultima atencion_x', 'Profesional Ultima atencion_x'], axis=1, inplace=True)



In [185]:
df_merged[df_merged.duplicated(subset=['RUT'], keep=False)]

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion
209,Cancer de Pulmón,4363827,0,ADRIANA DE LAS MERCE CATALÁN PÉREZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",0923937838/934687684/976540084.0,08-05-2024,Médico JOSE RAMON SILVA FIGUEROA CONSULTA ABRE...
210,Cancer de Pulmón,4363827,0,ADRIANA DE LAS MERCE CATALÁN PÉREZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",0923937838/934687684/976540084.0,08-05-2024,Médico JOSE RAMON SILVA FIGUEROA CONSULTA ABRE...
211,Cáncer de Pulmón . {Decreto Nro 22/2019},4363827,0,"CATALÁN PÉREZ, ADRIANA DE LAS MERCE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22819726 / 22819726 / 76540084 / 22819...,08-05-2024,Médico JOSE RAMON SILVA FIGUEROA CONSULTA ABRE...
212,Cáncer de Pulmón . {Decreto Nro 22/2019},4363827,0,"CATALÁN PÉREZ, ADRIANA DE LAS MERCE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22819726 / 22819726 / 76540084 / 22819...,08-05-2024,Médico JOSE RAMON SILVA FIGUEROA CONSULTA ABRE...
231,Cancer de Pulmón,4523076,7,LAURA ROSA ESPINOZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios o del pulmon, ...",74987712/74987712/-,07-06-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
232,Cancer de Pulmón,4523076,7,LAURA ROSA ESPINOZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios o del pulmon, ...",74987712/74987712/-,07-06-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
233,Cáncer de Pulmón . {Decreto Nro 22/2019},4523076,7,"ESPINOZA , LAURA ROSA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 26069132 / / 74987712 / - / -/7498771...,07-06-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
234,Cáncer de Pulmón . {Decreto Nro 22/2019},4523076,7,"ESPINOZA , LAURA ROSA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 26069132 / / 74987712 / - / -/7498771...,07-06-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
333,Cancer de Pulmón,5121321,1,MARIO FERNANDO FIGUEROA RAMOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],927388556/947300101/-,19-06-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...
334,Cancer de Pulmón,5121321,1,MARIO FERNANDO FIGUEROA RAMOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],927388556/947300101/-,19-06-2024,Médico JOSE RAMON SILVA FIGUEROA Control - Con...


In [186]:
df_merged = df_merged.drop_duplicates(subset=['RUT'], keep='first')


In [187]:
df_merged

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Pulmón,10024287,7,ISABEL ALEJANDRA DONOSO MEDRANO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],0981550027/0975219148/-,20-10-2023,Médico EDGARDO JAVIER OSSANDÓN MOLINA CONSULTA...
1,Cancer de Pulmón,10059878,7,CARLOS HERNÁN VILLEGAS ZORRICUETA,Alta,Fallecido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[TU PULMONAR],-/-/-,NaN,NaN
2,Cancer de Pulmón,10088381,3,VÍCTOR JOSÉ MARTÍNEZ CUEVAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SARCOMA EWING, TUMOR DE HUESO PÚBICO CON METÁ...",-/-/-,NaN,NaN
3,Cancer de Pulmón,10097136,4,EDUARDO ALBERTO NOHALES LEBRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",941057839/-/-,27-03-2024,Médico PATRICIA MARISOL ARIAS CAMPOVERDE CONSU...
4,Cancer de Pulmón,10107931,2,MALAQUIA RAMON LEAL AGUILAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,Cancer de Pulmón,9792505,4,DAVID ALEJANDRO ESTRELLA ROSALES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Hallazgos anormales en diagnostico por imagen...,0/0/-,06-11-2023,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...
811,Cancer de Pulmón,9869336,K,MARÍA JIMENA ROJAS PAREDES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",56998424688/56957386942/-,NaN,NaN
812,Cancer de Pulmón,9917387,4,ÁLVARO ANTONIO CHACÓN LUENGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],0993112390/02962398593/-,NaN,NaN
813,Cancer de Pulmón,9962036,6,FRANCISCO ANTONIO SÁEZ GÓMEZ,Alta,Fallecido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de los bronquios y del pulmon],-/-/-,NaN,NaN


# Crear Citas futuras

In [188]:

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)



ruta_del_archivo = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','AgendasFuturas', f'{today.year}{month_formatted}{day_formatted}_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)

In [189]:
# Mover los nombres de las columnas una posición hacia la izquierda
new_columns = list(dataframe_fu.columns)
new_columns = new_columns[1:] + [new_columns[0]]
dataframe_fu.columns = new_columns


In [190]:

dataframe_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1713881769526300695,DR. MATÍAS JOSÉ BAGOLINI FUENTES,Oftalmología,26-06-2024,1420,Anulada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0995765324,22827782,223147046,NaN,paraguaY,La Florida,0995765324,22827782,223147046,NaN
1713881769526300695,DR. MATÍAS JOSÉ BAGOLINI FUENTES,Oftalmología,26-06-2024,1420,Anulada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0995765324,22827782,223147046,NaN,paraguaY,La Florida,0995765324,22827782,223147046,NaN
1717602516653310166,DR. DANIEL ALBERTO CERDA RAMOS,Traumatología Adulto,26-06-2024,1300,Agendada,Control postquirurgico,CEX,N,N,NaN,...,72938996,NO ESPECIFICADO,NaN,NaN,Grumete Vargas,La Florida,72938996,NO ESPECIFICADO,NaN,NaN
1717602516653310166,DR. DANIEL ALBERTO CERDA RAMOS,Traumatología Adulto,26-06-2024,1300,Agendada,Control postquirurgico,CEX,N,N,NaN,...,72938996,NO ESPECIFICADO,NaN,NaN,Grumete Vargas,La Florida,72938996,NO ESPECIFICADO,NaN,NaN
1717603179188310480,Sala I-3,Imagenología AMB,26-06-2024,1610,Agendada,Rx EEII AP,CEX,N,N,NaN,...,72938996,NO ESPECIFICADO,NaN,NaN,Grumete Vargas,La Florida,72938996,NO ESPECIFICADO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719253531099500466,EU. SAMANTHA SOFÍA JORQUERA DROGUETT,Enfermería Multiespecialidad,24-12-2024,1130,Agendada,Control Hormonoterapia,CEX,N,N,NaN,...,0964744401,968626100,NaN,NaN,las gardenias,La Florida,0964744401,968626100,NaN,NaN
1719252323350500239,TOMA DE MUESTRA,Enfermería,24-12-2024,900,Agendada,Genérica toma de muestras,CEX,N,N,NaN,...,66616597,66616697,NaN,NaN,VIZCONDE,La Florida,66616597,66616697,NaN,NaN
1719252614545500288,EU. SAMANTHA SOFÍA JORQUERA DROGUETT,Enfermería Multiespecialidad,24-12-2024,1200,Agendada,Control Hormonoterapia,CEX,N,N,NaN,...,961664188,0225456859,0241591701,NaN,PIELAGO DEL SUR,La Florida,961664188,0225456859,0241591701,NaN
1719252323350500239,TOMA DE MUESTRA,Enfermería,24-12-2024,900,Agendada,Genérica toma de muestras,CEX,N,N,NaN,...,66616597,66616697,NaN,NaN,VIZCONDE,La Florida,66616597,66616697,NaN,NaN


In [191]:
# Eliminar duplicados por índice y mantener el primero
df_sin_duplicados = dataframe_fu.drop_duplicates(keep='first')


In [192]:
df_sin_duplicados['ESTADO_CITA']

1713881769526300695     Anulada
1717602516653310166    Agendada
1717603179188310480    Agendada
1717603121311310452    Agendada
1717603154203310467    Agendada
                         ...   
1718977794502500100    Agendada
1718981658351510235    Agendada
1719253531099500466    Agendada
1719252323350500239    Agendada
1719252614545500288    Agendada
Name: ESTADO_CITA, Length: 27525, dtype: object

In [193]:
df_sin_duplicados['ESTADO_CITA'].unique()

array(['Anulada', 'Agendada', 'En buzón de reasignación'], dtype=object)

In [194]:
df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA']=='Imagenología AMB']['DESCRIPCION']

1717603179188310480                                  Rx EEII AP
1717603121311310452                          Rx Rodilla Der F-L
1717603154203310467                          Rx Rotulas Axiales
1717603073162310438                          Rx Rodilla Izq F-L
1715948195763300854                                  Rx EEII AP
                                         ...                   
1718811584748500511                          Rx Col. Lumbosacra
1712323640596910122                    Rx Cadera Der Crosstable
1712323464883910069    Rx Pelvis AP para evaluación de Prótesis
1716830805648300587                                Rx Pelvis AP
1703877427182900912               TC de Abdomen y Pelvis con MC
Name: DESCRIPCION, Length: 1084, dtype: object

In [195]:
list(df_sin_duplicados)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY']

In [196]:
# Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
endocrinologia_df_fu = df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA'].isin(['Cirugía Toráx Adulto', 'Broncopulmonar Adulto','Imagenología AMB'])]
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
endocrinologia_df_fu = endocrinologia_df_fu[~endocrinologia_df_fu['ESTADO_CITA'].isin(['Anulada', 'En buzón de reasignación'])]

In [197]:
list(endocrinologia_df_fu.columns)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY']

In [198]:
# Convertir la columna de fecha a formato datetime
#endocrinologia_df_fu['CITAFECH'] = pd.to_datetime(endocrinologia_df_fu['CITAFECH'])

# Formatear la columna de fecha en el formato deseado
#endocrinologia_df_fu['CITAFECH'] = endocrinologia_df_fu['CITAFECH'].dt.strftime('%Y-%m-%d')

In [199]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1717603179188310480,Sala I-3,Imagenología AMB,26-06-2024,1610,Agendada,Rx EEII AP,CEX,N,N,NaN,...,72938996,NO ESPECIFICADO,NaN,NaN,Grumete Vargas,La Florida,72938996,NO ESPECIFICADO,NaN,NaN
1717603121311310452,Sala I-3,Imagenología AMB,26-06-2024,1150,Agendada,Rx Rodilla Der F-L,CEX,N,N,NaN,...,72938996,NO ESPECIFICADO,NaN,NaN,Grumete Vargas,La Florida,72938996,NO ESPECIFICADO,NaN,NaN
1717603154203310467,Sala I-3,Imagenología AMB,26-06-2024,1600,Agendada,Rx Rotulas Axiales,CEX,N,N,NaN,...,72938996,NO ESPECIFICADO,NaN,NaN,Grumete Vargas,La Florida,72938996,NO ESPECIFICADO,NaN,NaN
1717603073162310438,Sala I-3,Imagenología AMB,26-06-2024,1140,Agendada,Rx Rodilla Izq F-L,CEX,N,N,NaN,...,72938996,NO ESPECIFICADO,NaN,NaN,Grumete Vargas,La Florida,72938996,NO ESPECIFICADO,NaN,NaN
1715351598752310174,KLGO. ANAÍS CASTRO ALFARO,Broncopulmonar Adulto,26-06-2024,1530,Agendada,Grupo3 Esp Pre/Post B2 +DLCO + VPPlestismogr AMB,CEX,N,N,NaN,...,66753425,23113009,NaN,NaN,VOLCAN HORNOPIR,La Florida,66753425,23113009,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716830805648300587,Sala I-3,Imagenología AMB,12-11-2024,1100,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,090000000000,NaN,NaN,NaN,NaN,La Florida,090000000000,NaN,NaN,NaN
1703877427182900912,TAC 2,Imagenología AMB,02-12-2024,900,Agendada,TC de Abdomen y Pelvis con MC,HOS,N,N,NaN,...,958335023,99411984,NaN,NaN,VOLCAN LASTARRIA,La Florida,958335023,99411984,NaN,NaN
1718825753354500431,KLGO. TANNIA DENISSE MATAMALA CHACANA,Broncopulmonar Adulto,02-12-2024,900,Agendada,Control Kinesiología,CEX,N,N,NaN,...,09965613969,NaN,NaN,NaN,ayax,La Florida,09965613969,NaN,NaN,NaN
1718293899858510782,KLGO. TANNIA DENISSE MATAMALA CHACANA,Broncopulmonar Adulto,13-12-2024,1215,Agendada,Grupo1 Esp Basal + DLCO Est .cap Dif AMB,CEX,N,N,NaN,...,56996981069,949541176,NaN,NaN,santa raquel,La Florida,56996981069,949541176,NaN,NaN


In [200]:
endocrinologia_df_fu['DV'] = endocrinologia_df_fu['DOCUMENTO'].str[-1]

endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['DOCUMENTO'].str[:-1]

In [201]:
endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']

1717603179188310480    RUN
1717603121311310452    RUN
1717603154203310467    RUN
1717603073162310438    RUN
1715351598752310174    RUN
                      ... 
1716830805648300587    RUN
1703877427182900912    RUN
1718825753354500431    RUN
1718293899858510782    RUN
1718293950304510790    RUN
Name: NOMBRE_TIPO_DOCUMENTO, Length: 1865, dtype: object

In [202]:
endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']=='RUN']

In [203]:
endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['RUT'].astype(int)

In [204]:
endocrinologia_df_fu = endocrinologia_df_fu[['RUT','NOMBRE_AGENDA','ESPECIALIDAD_AGENDA','CITAFECH','CITAHORA','ESTADO_CITA','DESCRIPCION','ORIGEN','GES_CITA','GES_SOL','PATOLOGIA_GES_SOL','PACIENTE','CITA_CODUTEN','DOCUMENTO']]

In [205]:
endocrinologia_df_fu

,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
1717603179188310480,4189959,Sala I-3,Imagenología AMB,26-06-2024,1610,Agendada,Rx EEII AP,CEX,N,N,NaN,SERGIO SEGUNDO GARRIDO RIQUELME,AC00014620,4189959K
1717603121311310452,4189959,Sala I-3,Imagenología AMB,26-06-2024,1150,Agendada,Rx Rodilla Der F-L,CEX,N,N,NaN,SERGIO SEGUNDO GARRIDO RIQUELME,AC00014620,4189959K
1717603154203310467,4189959,Sala I-3,Imagenología AMB,26-06-2024,1600,Agendada,Rx Rotulas Axiales,CEX,N,N,NaN,SERGIO SEGUNDO GARRIDO RIQUELME,AC00014620,4189959K
1717603073162310438,4189959,Sala I-3,Imagenología AMB,26-06-2024,1140,Agendada,Rx Rodilla Izq F-L,CEX,N,N,NaN,SERGIO SEGUNDO GARRIDO RIQUELME,AC00014620,4189959K
1715351598752310174,4221399,KLGO. ANAÍS CASTRO ALFARO,Broncopulmonar Adulto,26-06-2024,1530,Agendada,Grupo3 Esp Pre/Post B2 +DLCO + VPPlestismogr AMB,CEX,N,N,NaN,MARÍA ANGÉLICA CONCHA,AC00014834,42213993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716830805648300587,28119064,Sala I-3,Imagenología AMB,12-11-2024,1100,Agendada,Rx Pelvis AP,CEX,N,N,NaN,BENJAMIN EXEQUIEL JARA CLAVERIA,AC00519708,281190644
1703877427182900912,6830435,TAC 2,Imagenología AMB,02-12-2024,900,Agendada,TC de Abdomen y Pelvis con MC,HOS,N,N,NaN,MARÍA TERESA GÓMEZ SOTO,AC00189407,68304350
1718825753354500431,6821537,KLGO. TANNIA DENISSE MATAMALA CHACANA,Broncopulmonar Adulto,02-12-2024,900,Agendada,Control Kinesiología,CEX,N,N,NaN,DANIEL SEGUNDO SILVA GONZÁLEZ,AC00041528,68215374
1718293899858510782,5480800,KLGO. TANNIA DENISSE MATAMALA CHACANA,Broncopulmonar Adulto,13-12-2024,1215,Agendada,Grupo1 Esp Basal + DLCO Est .cap Dif AMB,CEX,N,N,NaN,RAÚL ALBERTO LOYOLA CANCINO,AC00480104,54808003


In [206]:
# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)


In [207]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(endocrinologia_df_fu, index= False, header = True):
    ws.append(r)

for cell in ws['D'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_pulmon.xlsx'
wb.save(os.path.join(dir_actual, '..','CA PULMONAR',f'{nombre_archivo}'))